In [10]:
import torch
from torch.utils.data import Dataset, IterableDataset, DataLoader
import torch.nn as nn
import random
import numpy as np
import math
from tqdm.auto import tqdm
import torchmetrics

In [11]:
experiment_config = {
    'obj_size': 2,
    'obs_size': 2,
}
model_config = {
    'd_model': 128,
    'num_layers': 4,
    'num_obj': 2,
}
training_config = {
    'batch_size':1024,
    'lr': 1e-5,
    'max_step': 1000
}

In [12]:
device = torch.device('cuda')

In [24]:
class SymbolicWorldIterableDataset(IterableDataset):

    def __init__(self, num_objects, observation_size):
        self.num_objects = num_objects
        self.observation_size = observation_size
        pass

    def generate_observation(self):
        observation = torch.randint(0, self.num_objects, (self.observation_size,))

        parity_1 = ((observation == 1).sum())%2
        return {'observation':observation, 'parity_1':parity_1}

    def __iter__(self):
        while(True):
            yield self.generate_observation()

iterable_dataset = SymbolicWorldIterableDataset(experiment_config['obj_size'], experiment_config['obs_size'])
loader = DataLoader(iterable_dataset, batch_size=training_config['batch_size'])

test_loaders_list = []
for observation_size in [4,8,16,32]:
    test_loader_name = f'observation_size_{observation_size}'
    test_loader_data = DataLoader(SymbolicWorldIterableDataset(experiment_config['obj_size'], observation_size),\
        batch_size=training_config['batch_size'])
    test_loaders_list.append({
        'name': test_loader_name,
        'data_loader': test_loader_data
    })

if True:
    for a in loader:
        print(a['observation'][0])
        print(a['parity_1'][0])
        break


tensor([1, 0])
tensor(1)


In [25]:
class MySimpleModel(nn.Module):
    
    def __init__(self, model_config):
        super(MySimpleModel, self).__init__()
        d_model = model_config['d_model']
        n_layers = model_config['num_layers']
        n_objects = model_config['num_obj']
        n_clss = 5
        self.d_model = d_model
        self.rnn = nn.LSTM(d_model, d_model, n_layers, batch_first = True)
        self.token_embedding = torch.nn.Embedding(n_objects, d_model)
        self.clss_embedding = torch.nn.Embedding(n_clss, d_model)
        self.fc1 = torch.nn.Linear(d_model, 2)
        
    def forward(self, x, clss):
        '''
        x : long tensor: [batch, seq_len]
        clss: long tensor: [batch, 1]
        return: tensor: [batch, 2]
        '''
        temp = x
        temp = self.token_embedding(temp)
        clss_embedding = self.clss_embedding(clss)
        temp = torch.cat((clss_embedding, temp), dim=1)
        temp = self.rnn(temp)[0]
        temp = temp[:,-1,:]
        temp = self.fc1(temp)
        return temp

In [26]:
model = MySimpleModel(model_config).to(device)
loss_module = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=training_config['lr'])
f1_module = torchmetrics.F1Score(num_classes=2).to(device)

In [27]:
def eval(test_loader):
    with torch.no_grad():
            test_f1s = []
            for j, batch in enumerate(test_loader):
                x = batch['observation'].to(device)
                clss = torch.ones((x.shape[0], 1)).long().to(device)
                y = batch['parity_1'].to(device)
                output = model(x, clss)
                preds = output.argmax(dim=1)
                test_f1s.append(f1_module(preds, y).item())
                if j > 10:
                    break
            f1 = sum(test_f1s)/len(test_f1s)
            return round(f1, 4)

In [ ]:
for obs_size in [8,4,12]:
    print('obs_size: ', obs_size)
    iterable_dataset = SymbolicWorldIterableDataset(experiment_config['obj_size'], obs_size)
    loader = DataLoader(iterable_dataset, batch_size=training_config['batch_size'])

    loss_history = []
    max_steps = training_config['max_step']
    max_steps = 400
    log_every_steps = 200
    for i, batch in tqdm(enumerate(loader), total=max_steps, leave=False):
        x = batch['observation'].to(device)
        clss = torch.ones((x.shape[0], 1)).long().to(device)
        y = batch['parity_1'].to(device)
        optimizer.zero_grad()
        output = model(x, clss)
        loss = loss_module(output, y)
        loss_history.append(loss.item())
        loss.backward()
        optimizer.step()
        if i > max_steps:
            break
        if i % log_every_steps == 0 :
            print("step {}".format(i))
            avg_loss = sum(loss_history)/len(loss_history)
            print("loss: {:.2e}".format(avg_loss))
            loss_history = []
            train_f1 = f1_module(output.argmax(dim=1), y).item()
            print('train f1: ', round(train_f1,4))
            for test_loader in test_loaders_list:
                test_acc = eval(test_loader['data_loader'])
                print(test_loader['name'], " f1_score: " , test_acc)
            print('------------------------')
            if avg_loss < 1e-8:
                break